# Sunburst plot - Visualising Foreign Philanthropy Inflows to India
This has been tested with `Donors 2018-19.csv` from the CSIP "UPDATED OCTOBER 2020: Foreign Philanthropy (FCRA) dataset".

This dataset can be downloaded from here: https://csip.ashoka.edu.in/estimating-philanthropic-capital-india-data/

In [ ]:
import os

import numpy as np
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from google.colab import files


COLOURS = [
    'rgb(102, 197, 204)', 'rgb(246, 207, 113)', 'rgb(248, 156, 116)', 'rgb(220, 176, 242)', 'rgb(135, 197, 95)',
    'rgb(158, 185, 243)', 'rgb(254, 136, 177)', 'rgb(201, 219, 116)', 'rgb(139, 224, 164)', 'rgb(180, 151, 231)',
    'rgb(179, 179, 179)',
]

## Upload a file
Once you have downloaded and uzipped the data set, you should be able to upload the `Donors 2018-19.csv` file in the next step for use in the visualisation. This can take a while.

In [ ]:
uploaded = files.upload()

filename = list(uploaded.keys())[0]

## Create a Pandas dataframe
Convert the csv file in to a Pandas dataframe and have a quick look at the data.

In [ ]:
# Use the file name from the last upload, or modify for previously uploaded files.
df = pd.read_csv(filename)

print(filename)
df

In [ ]:
n = df.nunique(axis=0) 
  
print("No.of.unique values in each column :\n", n)

## Create the plot data

In [ ]:
data = {}
total = 0

# Extract the data for each 'purpose' type (i.e. Social, Educational etc.)
for i, row in df.iterrows():

    # Only look at Institutional donors for now
    if row['Donor_type'] != 'Institutional':
        continue

    purpose = row.Purpose
    state = row.State
    district = row.District

    # Make the amounts in billions
    amount = row['Amount (INR)'] / 1e9
    total += amount

    if purpose in data:
        data[purpose]['count'] += amount
    else:
        data[purpose] = {'count': amount, 'states': {}} 

    if state in data[purpose]['states']:
        data[purpose]['states'][state]['count'] += amount
    else:
        data[purpose]['states'][state] = {'count': amount, 'districts': {}}
    
    if district in data[purpose]['states'][state]['districts']:
        data[purpose]['states'][state]['districts'][district]['count'] += amount
    else:
        data[purpose]['states'][state]['districts'][district] = {'count': amount}


# Build the sunburst plot source data
source_data = {}

for purpose in data.keys():
    
    source_data[purpose] = []
    source = source_data[purpose]

    purpose_total = data[purpose]['count']
    percent_of_total = (purpose_total / total) * 100

    initial_label = f'{purpose}<br>{purpose_total:.2f} billion (INR)<br>({percent_of_total:.2f}%)'

    source.append(
        ('', initial_label, purpose_total),
    )

    for state, s_data in data[purpose]['states'].items():
        state_total = s_data['count']

        source.append(
            (initial_label, state, state_total),
        )
        
        for district, d_data in s_data['districts'].items():
            district_total = d_data['count']
            source.append(
                (state, f' {district} ', district_total),
            )

## Build the plots

In [ ]:
# Set the number of subplot rows/cols
rows = 2
cols = 3

keys = list(data.keys())

fig = make_subplots(
    rows=rows, 
    cols=cols,
    specs = [[{'type': 'sunburst'} for c in np.arange(cols)] for r in np.arange(rows)],
    vertical_spacing=0.04,
    horizontal_spacing=0.04,
    subplot_titles=keys,
)

for i, key in enumerate(keys):
    
    source = source_data[key]
    parents, labels, values = zip(*source)

    row = i // cols + 1
    col = i % cols + 1

    # Calculate the percentage of total for each slice to be used in the hover labels
    percents = (np.array(values) / total) * 100

    fig.add_trace(
        go.Sunburst(
            labels=labels,
            parents=parents,
            values=values,
            branchvalues="total",
            marker_line_width=0.1,
            marker_line_color='rgba(0, 0, 0, 0.9)',
            customdata=percents,
            hovertemplate='<b>%{label}</b><br>Amount: %{value:.2f} billion (INR)<br>Percent of total: %{customdata:.4f}%<extra></extra>',
        ),
        row=row,
        col=col,
    )

# Update subplot title size
for annotation in fig['layout']['annotations']:
    annotation['font_size'] = 13

# Update the main layout and add title
fig.update_layout(
    title=f'Distribution of institutional donations across states and districts - {total:.0f} billion (INR) [{filename}]',
    title_x=0.5,
    height=1000,
    margin = dict(t=120, l=20, r=20, b=20),
    colorway=COLOURS,
)

fig.show()